In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [2]:
states=pd.read_csv("C:/Users/Gunesim/Desktop/Ödevler/states_all.csv", engine="python", encoding="utf8")

1NCİ SORU:

In [3]:
states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

In [4]:
states.isnull().sum()*100/states.shape[0]

PRIMARY_KEY                      0.000000
STATE                            0.000000
YEAR                             0.000000
ENROLL                          17.627346
TOTAL_REVENUE                   14.209115
FEDERAL_REVENUE                 14.209115
STATE_REVENUE                   14.209115
LOCAL_REVENUE                   14.209115
TOTAL_EXPENDITURE               14.209115
INSTRUCTION_EXPENDITURE         14.209115
SUPPORT_SERVICES_EXPENDITURE    14.209115
OTHER_EXPENDITURE               17.627346
CAPITAL_OUTLAY_EXPENDITURE      14.209115
GRADES_PK_G                     11.595174
GRADES_KG_G                      8.847185
GRADES_4_G                       8.780161
GRADES_8_G                       8.780161
GRADES_12_G                      8.780161
GRADES_1_8_G                     8.780161
GRADES_9_12_G                    8.780161
GRADES_ALL_G                    11.595174
AVG_MATH_4_SCORE                64.075067
AVG_MATH_8_SCORE                64.343164
AVG_READING_4_SCORE             64

2NCİ SORU:
Boş değer olan satırlar çıkarıldığında geriye 413 satır kalıyor. Bu da toplam satırların %27'sine denk geliyor ve önemli miktarda veri kayboluyor.

Tüm değerler sayısal olduğu için ortalama değerlerle doldurulabilir.

3NCÜ SORU (Tablodaki boş değerlerin her yıla ait ortalama değerleriyle doldurulması):

In [5]:
#Boş Değerler Tablosu (her döngüde tekrarlayan şekilde ortalama hesaplamamak için)
yil_bos=states.groupby("YEAR").mean()

In [6]:
#Tablodaki boş değerleri ortalama değerlerle doldurma
for sutun in states.columns[3:,]:
    for yil in states["YEAR"].unique():
        states[sutun][(states["YEAR"]==yil) & (states[sutun].isnull())]=yil_bos[sutun][yil]

Tüm değerler ortalama ile doldurulamamaktadır.

4NCÜ SORU (Tabloda ortalamalardan sonra boş kalan değerlerin enterpolasyon ile doldurulması):

In [7]:
#Tabloda boş olan değerleri enterpolasyon ile doldurma
for sutun in states.columns[3:,]:
    for yil in states["YEAR"].unique():
        
        #Ortalamalar tablosundaki en küçük değeri doldurma
        if (yil==states["YEAR"].unique().min()) & (pd.isnull(yil_bos[sutun][yil])):
            sayi=0
    
            while pd.isnull(yil_bos[sutun][yil+1+sayi]) | pd.isnull(yil_bos[sutun][yil+2+sayi]):
                sayi+=1
            
            while sayi>=0:
                yil_bos[sutun][yil+sayi]=2*yil_bos[sutun][yil+1+sayi]-yil_bos[sutun][yil+2+sayi]
                sayi-=1

        #Ortalamalar tablosundaki en büyük değeri doldurma
        elif (yil==states["YEAR"].unique().max()) & (pd.isnull(yil_bos[sutun][yil])):
            sayi=0
        
            while pd.isnull(yil_bos[sutun][yil-1-sayi]) | pd.isnull(yil_bos[sutun][yil-2-sayi]):
                sayi+=1
            
            while sayi>=0:
                yil_bos[sutun][yil-sayi]=2*yil_bos[sutun][yil-1-sayi]-yil_bos[sutun][yil-2-sayi]
                sayi-=1

         #Ortalamalar tablosunda aralarda kalan değeri doldurma
        elif pd.isnull(yil_bos[sutun][yil]):
            sayi=0
            
            while (pd.isnull(yil_bos[sutun][yil+1+sayi])) & (yil+1+sayi<states["YEAR"].unique().max()):
                sayi+=1
                
            fark=0
            fark=(yil_bos[sutun][yil+1+sayi]-yil_bos[sutun][yil-1])/(sayi+2)

            while sayi>=0:
                yil_bos[sutun][yil+sayi]=yil_bos[sutun][yil+1+sayi]-fark
                sayi-=1
                
for sutun in states.columns[3:,]:
    for yil in states["YEAR"].unique():
        states[sutun][(states["YEAR"]==yil) & (states[sutun].isnull())]=yil_bos[sutun][yil]

5NCİ SORU:

> 2nci Soruda; çok fazla eksik veri olduğu için, bu şekilde çalışmak analizimizin doğruluğunu azaltabilir. Verilerimizde boş değer az olduğunda uygundur.

> 3ncü Soruda; boş değerleri yıllara ait ortalama değerlerle doldurmak uygun olabilir. Ancak, yine de bazı yıllar için ortalama değerler de eksik. Halâ eksik veri kalıyor. Düzensiz veriler için uygundur.

> 4ncü Soruda; enterpolasyon yaptığımızda verilerin seyrini önemli ölçüde etkileme riski mevcut. Ancak, verilerde bir trend söz konusu ise ortalama ile doldurmaktan daha anlamlı olabilir. Trend olan veriler için uygundur.